### 절대 모멘텀
- 파생변수 'STD-YM' 생성 -> index에서 년-월 추출하여 대입
- 'STD-YM' 별 마지막날의 데이터를 month_last_df 저장
- 전월의 수정주가 값을 가진 파생변수를 생성 
- 전년도의 수정주가 값을 가진 파생변수를 생성
- 전월의 데이터와 전년도의 데이터를 기준으로 하여 거래 내역 생성
- 수익율 계산

In [ ]:
import pandas as pd 
import numpy as np 
from datetime import datetime

In [ ]:
df = pd.read_csv("../../csv/MSFT.csv", index_col='Date')

In [ ]:
df.head(1)

In [ ]:
# index를 시계열로 변경 
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

In [ ]:
# index의 데이터에서 년-월 추출하여 새로운 파생변수 STD-YM에 대입
df['STD-YM'] = df.index.strftime('%Y-%m')

In [ ]:
df.iloc[10:20]

In [ ]:
# 월의 말일의 조건식
# 다음 행의 STD-YM과 오늘의 STD-YM의 값이 다른경우
flag = df['STD-YM'] != df.shift(-1)['STD-YM']
df.loc[flag,]

In [ ]:
# 필터를 이용하여 마지막날의 데이터를 추출 
_list =  df['STD-YM'].unique()

month_last_df = pd.DataFrame()

for i in _list:
    flag = df['STD-YM'] == i
    data = df.loc[flag,].tail(1)
    month_last_df = pd.concat([month_last_df, data], axis=0)

In [ ]:
month_last_df

In [ ]:
# 전월의 수정 주가를 BF_1M 파생변수에 대입 
# 전년도의 수정주가를 BF_12M 파생변수에 대입 
# 두개의 파생변수는 결측치는 0으로 대체
month_last_df['BF_1M'] = month_last_df['Adj Close'].shift(1).fillna(0)
month_last_df['BF_12M'] = month_last_df['Adj Close'].shift(12).fillna(0)

- 거래 내역 생성 
    - (전월의 수정주가 / 전년도의 수정주가) - 1 의 값이 0보다 크고 무한대가 아닌 경우

In [ ]:
df['trade'] = ""

for i in month_last_df.index:
    signal = ""

    # 절대 모멘텀 계산식 
    momentum_index = month_last_df.loc[i, 'BF_1M'] /\
        month_last_df.loc[i, 'BF_12M'] - 1
    flag = True if ( (momentum_index > 0) & (momentum_index != np.inf) ) \
        else False
    
    if flag:
        signal = "buy"
    
    print(f"날짜 : {i}, 모멘텀 인덱스 : {momentum_index}, flag : {flag}, signal : {signal}")
    df.loc[i:, 'trade'] = signal


In [ ]:
df['trade'].value_counts()

In [ ]:
df['rtn'] = 1
for i in df.index:
    # 매수 조건식 (전날의 trade가 "" 오늘의 trade가 "buy")
    if (df.shift().loc[i, 'trade'] == "") & (df.loc[i, 'trade'] == "buy"):
        buy = df.loc[i, 'Adj Close']
        print(f"매수일 : {i}, 매수가 : {buy}")
    # 매도 조건식 (전날의 trade가 "buy" 오늘의 trade가 "")
    elif (df.shift().loc[i, 'trade'] == "buy") & (df.loc[i, 'trade'] == ""):
        sell = df.loc[i, 'Adj Close']
        rtn = sell / buy
        df.loc[i, 'rtn'] = rtn
        print(f"매도일 : {i}, 매도가 : {sell}, 수익율 : {rtn}")

In [ ]:
df['acc_rtn'] = df['rtn'].cumprod()

In [ ]:
df.tail()

### 절대 모멘텀 함수화
1. STD-YM 생성하는 함수 
    - 매개변수 2개 (데이터프레임, 기준이 되는 컬럼)
    - 데이터프레임의 복사본을 생성
    - 컬럼에 Date가 포함되어있는지 확인하고 컬럼에 Date 존재한다면 Date를 인덱스로 변환
    - 인덱스를 시계열 데이터로 변경 
    - 데이터중 결측치나 무한대 값을 제외 
    - 기준이 되는 컬럼을 제외한 나머지 컬럼은 모두 삭제
    - 'STD-YM' 컬럼을 생성하여 index에서 '년-월'데이터를 추출하여 대입 
    - 수정이 된 데이터프레임을 리턴

In [ ]:
def create_YM(
        _df, 
        _col = 'Adj Close'
):
    df = _df.copy()
    # Date가 컬럼에 포함되어있는가?
    if 'Date' in df.columns:
        # 포함되어있다면 Date를 인덱스로 변환 
        df.set_index('Date', inplace=True)
    # 인덱스를 시계열데이터로 변경
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
    # 결측치, 무한대 데이터를 제거 기준이되는 컬럼만 두고 나머지 모두 제거 
    flag = df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    df = df.loc[~flag, [_col]]
    # 파생변수 STD-YM 생성
    df['STD-YM'] = df.index.strftime('%Y-%m')

    return df


In [ ]:
df = pd.read_csv('../../csv/AAPL.csv')

In [ ]:
ym_df = create_YM(df)

- 두번째 함수 생성
    - 매개변수 5개 (데이터프레임, 시작시간, 종료시간, 모멘텀 기간, 기준시점)
    - 시작시간의 기본값은 2010-01-01
    - 종료시간의 기본값은 오늘 날짜
    - 모멘텀 기간의 기본값은 12
    - 기준시점의 기본값은 1이고 0으로 변경시 월초의 데이터로 구성 
    - 기준시점의 값에 따라 (월말|월초) 데이터만 모아서 새로운 데이터프레임 생성
    - 생성된 데이터프레임을 기준으로 BF1 컬럼을 생성하여 전월의 데이터를 대입
    - BF2 컬럼을 생성하여 모멘텀 기간(6 -> 6개월 전) 전의 데이터를 대입
    - 결측치는 0으로 대체
    - 데이터프레임을 시작시간과 종료시간으로 데이터 필터링 
    - 결과를 리턴

In [ ]:
def create_month(
        _df, 
        _start = "2010-01-01", 
        _end = datetime.now(), 
        _momentum = 12, 
        _select = 1
):
    if _select == 1:
        # 월말의 데이터들을 새로운 데이터프레임으로 생성 
        # 현재 행의 년-월과 다음 행의 년-월이 다른 경우 
        flag = _df['STD-YM'] != _df.shift(-1)['STD-YM']
        # df = _df.loc[flag,]
    elif _select == 0:
        flag = _df['STD-YM'] != _df.shift()['STD-YM']
        # df = _df.loc[flag,]
    else :
        return "_select 인자는 0과 1이 가능하다"
    col = _df.columns[0]
    df = _df.loc[flag,]
    df['BF1'] = df.shift()[col].fillna(0)
    df['BF2'] = df.shift(_momentum)[col].fillna(0)
    start = datetime.strptime(_start, '%Y-%m-%d')
    if type(_end) == "str":
        end = datetime.strptime(_end, '%Y-%m-%d')
    else:
        end = _end
    df = df.loc[start:end,]
    return df

In [ ]:
month_df = create_month(ym_df, _select=0)

- 세번째 함수 
    - 매개변수 3개 (1번 함수결과인 데이터프레임(_df1), 2번 함수의 결과인 데이터프레임(_df2), 모멘텀 스코어)
    - _df1에는 trade 컬럼을 생성 "" 대입
    - _df1에 rtn 컬럼을 생성 1 대입
    - _df2를 이용하여 momentum_index를 구하고 _df1에 거래 내역 대입
    - 거래 내역을 이용하여 수익율 계산
    - acc_rtn 컬럼을 생성 누적 수익율 대입 
    - _df1, 총 누적수익율 리턴


In [ ]:
def create_rtn(_df1, _df2, _score = 1):
    # _df1에 파생변수 2개 생성 
    _df1['trade'] = ""
    _df1['rtn'] = 1

    # _df2 데이터를 이용하여 momentum_index를 계산하고 거래 내역 추가 
    for i in _df2.index:
        signal = ""

        # 절대 모멘텀 계산
        momentum_index = _df2.loc[i, 'BF1'] / _df2.loc[i, 'BF2'] - _score

        # 모멘텀 인덱스가 무한대가 아니고 0보다 큰 경우 
        flag = (momentum_index > 0) & (momentum_index != np.inf)

        if flag:
            signal = 'buy'
        
        _df1.loc[i:, 'trade'] = signal
        print(f"날짜 : {i}, 모멘텀 인덱스 : {momentum_index}, flag : {flag}, signal : {signal}")
    # 수익율 계산
    col = _df1.columns[0]

    for i in _df1.index:
        # 구매한 날의 조건식 (전날의 trade가 "" 오늘의 trade가 "buy")
        if (_df1.shift().loc[i, 'trade'] == "") & (_df1.loc[i, 'trade'] == "buy"):
            buy = _df1.loc[i, col]
            print(f"매수일 : {i}, 매수가 : {buy}")
        # 판매한 날의 조건식 (전날의 trade가 "buy" 오늘의 trade가 "")
        elif (_df1.shift().loc[i, 'trade'] == "buy") & (_df1.loc[i, 'trade'] == ""):
            sell = _df1.loc[i, col]
            rtn = sell / buy
            _df1.loc[i, 'rtn'] = rtn
            print(f"매도일 : {i}, 매도가 : {sell}, 수익율 : {rtn}")
    # 누적수익율 계산
    _df1['acc_rtn'] = _df1['rtn'].cumprod()

    # 총 누적수익율 변수에 대입 
    acc_rtn = _df1.iloc[-1, ]['acc_rtn']

    return _df1, acc_rtn

In [ ]:
create_rtn(ym_df, month_df)

In [ ]:
import momentum as mm

In [ ]:
import yfinance as yf

In [ ]:
df = yf.download('TSLA', start='2010-01-01')
df

In [ ]:
ym_df = mm.create_YM(df)

In [ ]:
ym_df.head()

In [ ]:
month_df = mm.create_month(ym_df, _start='2015-01-01')
month_df.head()

In [ ]:
result, acc_rtn = mm.create_rtn(ym_df, month_df)

In [ ]:
acc_rtn